In [16]:
import jpy_tools.parseSnake as jps

In [17]:
snakefile = jps.SnakeMakeFile()

In [18]:
header = jps.SnakeHeader(snakefile, '/public/home/liuzj/scripts/pipeline/polyACallerBasedOnWp/snakemake/config.yaml')
header.generateContent()

config contents:

     resultDir      :/public/home/liuzj/scripts/pipeline/polyACallerBasedOnWp/results/

    pipelineDir     :/public/home/liuzj/scripts/pipeline/polyACallerBasedOnWp/scripts/

       rawSeq       :/public/home/liuzj/scripts/pipeline/polyACallerBasedOnWp/data/example.fa

       genome       :~/data/Araport11/genome.fa

       f5Dir        :/public/home/liuzj/projects/singleCell/00_data/01_rootUseData/01_basecalledData/00_basecalledData/all/workspace/

     f5Summary      :/public/home/liuzj/projects/singleCell/00_data/01_rootUseData/01_basecalledData/00_basecalledData/all/sequencing_summary.txt

      geneAnno      :~/data/Araport11/gene.bed



In [19]:
buildIndex = jps.SnakeRule(snakefile, 'buildIndex', 1, 56)
buildIndex.setInput(b=['rawSeq'])
buildIndex.setParams(b=['genome','f5Dir','f5Summary','geneAnno'])
buildIndex.setOutput(a=['exampleIndex.h5'])
buildIndex.setShell("""
python step1_buildIndex.py -i {input.rawSeq} --genome {params.genome} -t {threads} --f5dir {params.f5Dir} --f5summary {params.f5Summary} --bed {params.geneAnno} -o {output.exampleIndex}
""")
buildIndex.generateContent()

In [20]:
polyACaller = jps.SnakeRule(snakefile, 'polyACaller', 2, 56)
polyACaller.setInput(a=['exampleIndex.h5'])
polyACaller.setOutput(a=['exampleAddPolyALength.h5'])
polyACaller.setShell("""
python step2_polyACaller.py -i {input.exampleIndex} -o {output.exampleAddPolyALength} -t {threads}
""") 
polyACaller.generateContent()

In [21]:
ruleAll = jps.SnakeAll(snakefile)
ruleAll.generateContent(exampleAddPolyALength=0)

In [22]:
snakefile.generateContent('/public/home/liuzj/scripts/pipeline/polyACallerBasedOnWp/snakemake/snakefile')

configfile: "/public/home/liuzj/scripts/pipeline/polyACallerBasedOnWp/snakemake/config.yaml"
pipelineDir = config['pipelineDir']


rule all:
    input:
        exampleAddPolyALength = f"{config['resultDir']}step2_polyACaller/exampleAddPolyALength.h5"

rule buildIndex:
    input:
        rawSeq = config['rawSeq']
    output:
        exampleIndex = f"{config['resultDir']}step1_buildIndex/exampleIndex.h5"
    params:
        genome = config['genome'],
        f5Dir = config['f5Dir'],
        f5Summary = config['f5Summary'],
        geneAnno = config['geneAnno']
    threads:1
    shell:
        """
cd {pipelineDir}
python step1_buildIndex.py -i {input.rawSeq} --genome {params.genome} -t {threads} --f5dir {params.f5Dir} --f5summary {params.f5Summary} --bed {params.geneAnno} -o {output.exampleIndex}
        """

rule polyACaller:
    input:
        exampleIndex = f"{config['resultDir']}step1_buildIndex/exampleIndex.h5"
    output:
        exampleAddPolyALength = f"{config['resultDir']}step2_